In [ ]:
#Import Libraries
import pandas as pd 

import applescript
import os
import subprocess

In [ ]:
#inquire user of equipment that some gyms might not have
def has_equipment(df_filtered, df):
    
    equipment = ["Bands", "Kettlebells", "Cable", "Exercise Ball", "Medicine Ball", "E-Z Curl Bar", "Foam Roll"]
    
    for x in equipment:
        
        ask = input(f"Do you have {x} (y/n): ")
        
        if ask == 'n':
            df_filtered = df_filtered[df_filtered["Equipment"] != x]
            df = df[df["Equipment"] != x]
    
    return df_filtered, df

In [ ]:
#Filters out body parts that don't need to be worked on for the day
def PushPullLegs(df):
    global d1 

    d1 = {"push":["Shoulders", "Traps", "Chest", "Triceps"], "pull":["Middle Back", "Lower Back", "Lats", "Biceps", "Forearms"],
          "legs":["Glutes", "Hamstrings", "Quadriceps", "Calves"]}

    day_type = input("Is today push, pull, or legs? ")
    
    
    df2 = df[df["BodyPart"].isin(d1[day_type.lower()])]
    
    
    return df2, day_type

In [ ]:
#Workout schedule should output one workout per body part first, then randomly select body parts for the rest
# WANT one workout per body part  (one stretch per body part worked before workout)
# & 3 ab workouts after

def addStretches(df, day_type):
      
    final_df = pd.DataFrame(columns=df.columns)
    
    d1 = {"push":["Shoulders", "Traps", "Chest", "Triceps"], "pull":["Middle Back", "Lower Back", "Lats", "Biceps", "Forearms"],
          "legs":["Glutes", "Hamstrings", "Quadriceps", "Calves"]}

    
    for x in d1[day_type]:
          
            
          df2 = df[df["Type"]=="Stretching"]  
          df2 = df2[df2["BodyPart"] == x]
          
          #this if statement concates the randow dataframe row to the final one if df2 not empty after filtering  
          if not df2.empty:
                random_row = df2.sample(n=1)
                final_df = pd.concat([final_df, random_row], axis=0)  
                
                      
    return final_df  

In [ ]:
def addWorkout(df, final_df, day_type):
    
    d1 = {"push":["Shoulders", "Traps", "Chest", "Triceps"], "pull":["Middle Back", "Lower Back", "Lats", "Biceps", "Forearms"],
          "legs":["Glutes", "Hamstrings", "Quadriceps", "Calves"]}

    
    for x in d1[day_type]:
                
          df2 = df[df["Type"]!="Stretching"]  
          df2 = df2[df2["BodyPart"] == x]
          
          #this if statement concates the randow dataframe row to the final one if df2 not empty after filtering  
          if not df2.empty:
                random_row = df2.sample(n=1)
                final_df = pd.concat([final_df, random_row], axis=0) 
    
    return final_df

In [ ]:
def add_ab_workout(df, final_df):
    
    df2 = df[df["Type"]!="Stretching"]  
    df2 = df2[df2["BodyPart"] == "Abdominals"]
    
    if not df2.empty:
        random_row = df2.sample(n=3)
        final_df = pd.concat([final_df, random_row], axis=0)
                
                
    return final_df

In [ ]:
#Reading DataFrame
df = pd.read_csv("megaGymDataset.csv")

#Drop unneeded columns
df.drop(["Unnamed: 0", "RatingDesc"],axis=1,inplace=True)

In [ ]:
#Cleaning DataFrame Data 
#Filling in Nan Values
mean_ratings = df['Rating'].mean()


df['Rating'].fillna(round(mean_ratings,1),inplace=True)

description_fill = "No Description! Search up Online!"
df["Desc"].fillna(description_fill, inplace=True)



In [ ]:
#Main CELL where everything happens (later will be changed to main function)

level = input("Are you a Beginner (B), Intermediate (I), or an Expert (E)")#functionaility not yet implemented

#Delete Data from DataFrame depending on user's level/experience
if level == "B":
    df = df[(df["Level"] != "Expert") & (df["Level"] != "Intermediate")]
elif level == "I":
    df = df[df["Level"]!="Expert"]

workout_day = int(input("Push/Pull/Legs (0), Five Day Split (1), or Full Body (2): ")) #functionaility not yet implemented
    
#filter out DF of body parts not needed to be worked on a specific day
BodyPart_DF, day_type = PushPullLegs(df)

#filterd out DF of equipment that user does not have
equip_Sorted_df, new_df = has_equipment(BodyPart_DF, df)


#passes through filtered DF of Body Parts and Equipment
addStretchDF = addStretches(equip_Sorted_df, day_type)

# add workouts to the DataFrame
addWorkoutDF = addWorkout(equip_Sorted_df, addStretchDF, day_type)

#this one requires the equipment filter DF instead 
#Add ab workouts to DataFrame
final_df = add_ab_workout(new_df, addWorkoutDF)


In [ ]:
#Changing the index of the DF to be a workout order column
num_of_rows = final_df["BodyPart"].count()

new_index_list = list(range(1,num_of_rows+1))

final_df.insert(0, "Workout Order", new_index_list)

#final_df.index = new_index_list
final_df = final_df.reset_index(drop=True)

In [ ]:
final_df.to_html("myWorkoutPlan.html", index= False)  #html file preserves the table format in apple notes versus a csv file

In [ ]:
print("This is your workout schedule:\n")
display(final_df) #displays workout plan to user as a dataframe